 <div>
     <hr>
     <h1 style=" font-size: 32px; margin-bottom: 10px; text-transform: uppercase;">SHA-3</h1>
     <hr>
 </div>


### Importe des bibliothèques nécessaires

In [14]:
import numpy as np
import binascii

### Les parametres globales

In [20]:
#declaration des longueurs des blocks
output_len = [224, 256, 384, 512]

global w
w = 64

### Fonctions auxiliaires

#### 1. convertir un text ascii en binaire

In [3]:
def str_to_bin(s):
    # Convertir la chaîne en bytes
    byte_data = s.encode('utf-8')
    
    # Convertir les bytes en hexadécimal
    hex_data = binascii.hexlify(byte_data).decode('utf-8')
    
    # Convertir chaque caractère hexadécimal en binaire
    binary_data = ''.join(format(int(c, 16), '04b') for c in hex_data)
    
    return binary_data
    

#### 2. Convertir une chaîne binaire en hexadécimale

In [ ]:
def hex_digest(binary_string):
    
    # fixer la longueur pour etre un multiple de 8
    padding_length = (8 - len(binary_string) % 8)
    binary_string_padded = binary_string + '0' * padding_length
    
    # Convertir la chaîne binaire en un tableau d'octets
    byte_array = bytearray()
    for i in range(0, len(binary_string_padded), 8):
        byte = binary_string_padded[i:i+8]
        byte_array.append(int(byte, 2))
    
    # Convertir le tableau d'octets en une chaîne hexadécimale
    hex_string = binascii.hexlify(byte_array).decode('utf-8')
    
    return hex_string

In [16]:
#test
m = "Master CSI 2024"
msg_bin = str_to_bin(m)

<hr style="color: black; height: 1px;">

#### Exercice 1 : Padding d'un message pour SHA-3 (224, 256, 384, et 512) 

In [17]:
# Calcule et concatenation du padding simple
def simple_padd(ch_bits, block_size):
    #intialisons padding par  "1"
    padd = "1"
    #calcule nombre des bits a ajouter exemple(224 - (l%b) - 1)
    nb_bits = block_size - (len(ch_bits)%block_size) -1
    #concatener les zeros
    padd += "0"*nb_bits
    #concatener msg_binaire||padding
    ch_bits += padd
    
    return ch_bits

# Calcule et concatenation du padding 10*1
def second_padd(ch_bits, block_size):
    #ajoute 1 au debut du padding
    padd = "1"
    #calcule nombre des bits a ajouter exemple(b - l % b + 1)
    nb_bits = block_size - ( len(ch_bits) % block_size ) - 2
    #concatener les zeros
    padd += "0"*nb_bits
    #ajoute 1 a la fin du padding
    padd += "1"
    #concatener  msg_bianire||padding
    ch_bits += padd
    
    return ch_bits

In [ ]:
# Test
print(f"ASCII : {m} \nBinaire: {msg_bin}")
print("-"*100)
#affichage de padding simple
print(f"Padding simple de : {m}")
for b in output_len:
    print(f"Block size {b}")
    res = simple_padd(msg_bin,b)
    print(f"Massage avec padding de longueur {len(res)}:\n{res}\n")

#affichage de padding 10*1
print("-"*100)
print(f"Padding 10*1 de : {m}")
for b in output_len:
    print(f"Block size {b}")
    res = second_padd(msg_bin,b)
    print(f"Massage avec padding de longueur {len(res)}:\n{res}\n")

<hr style="color: black; height: 1px;">

#### Exercice 2:

Supposons qu'il existe une fonction de hachage $H$ sans padding qui est résistante aux collisions, c'est-à-dire qu'il est difficile de trouver deux messages distincts $m_1$ et $m_2$ tels que $H(m1)=H(m2)$.

Considérons un message $M$ de taille arbitraire. Posons un autre message $M'$ par exemple: $M'= M∥ p$ 

Puisque $H$ ne tient pas compte de la longueur du message, les messages $M$ et $M'$ sont traités de manière identique par $H$.

Ainsi : $H(M)=H(M')$

Cela contredit l'hypothèse de départ selon laquelle $H$ est résistante aux collisions.

<hr style="color: black; height: 1px;">

#### Exercice 3: Transforme une chaine des bits en un etat de SHA-3

In [5]:
#Convertit une chaîne de bits en état SHA3 (5x5xw).
def chBits_to_state(bits, w):
    state = np.zeros((5, 5, w), dtype=np.uint64)
    index = 0
    for z in range(w):
        for y in range(5):
            for x in range(5):
                if index < len(bits):
                    state[x, y, z] = int(bits[index])
                    index += 1
                else:
                    state[x, y, z] = 0
    return state

#test
#A = chBits_to_state(msg_bin,w)
#print("Etat A du message avec padding:")
#print(A)

#### Inverse transforme

In [6]:
# Trouver les coordonnees d'un element de S dans A
def state_to_chBits(state):
    bits = []
    for x in range(5):
        for y in range(5):
            for z in range(w):
                bits.append(str(state[x, y, z]))  # Convertir chaque bit en chaîne
    return ''.join(bits)  # Retourner une chaîne de bits

In [ ]:
#test
X = int(input("Entrez l'indice X dans S : "))

x, y, z = state_to_chBits(X)
print(f"Les coordonnées correspondant à S[{X}] sont : A[{x}, {y}, {z}]")

<hr style="color: black; height: 1px;">

#### Exercice 4: la fonction $\theta$

On a la formule de $\theta$ : $A[x,y,z] \leftarrow A[x,y,z] \oplus \left( \sum_{i=0}^4 A[x-1, i, z] \right) \oplus \left( \sum_{i=0}^4 A[x+1, i, z-1] \right)$

**Algorithme :**

1. **Calcul des sommes de colonnes :**
   Pour chaque $x$ et $z$, calculer : $  C[x, z] = \sum_{i=0}^4 A[x, i, z]$

2. **Calcul des différences :**
   Pour chaque $x$ et $z$, calculer : $   D[x, z] = C[(x-1)\mod 5, z] \oplus C[(x+1) \mod 5, (z-1) \mod w]$

3. **Mise à jour de $A[x, y, z]$ :**
   Pour chaque $x, y$ et $z$, mettre à jour : $  A[x, y, z] \leftarrow A[x, y, z] \oplus D[x, z]$

In [7]:
#La fonction Theta
def theta(A, w):
    #initialiser C et D par des zeros
    C = np.zeros((5, w), dtype=np.uint64)
    D = np.zeros((5, w), dtype=np.uint64)
    
    # Calcul des sommes de colonnes
    for x in range(5):
        for z in range(w):
            C[x, z] = A[x, 0, z] ^ A[x, 1, z] ^ A[x, 2, z] ^ A[x, 3, z] ^ A[x, 4, z]
    
    # Calcul des différences
    for x in range(5):
        for z in range(w):
            D[x, z] = C[(x - 1) % 5, z] ^ C[(x + 1) % 5, (z - 1) % w]
    
    # Mise à jour de l'état
    for x in range(5):
        for y in range(5):
            for z in range(w):
                A[x, y, z] ^= D[x, z]
    
    return A

<hr style="color: black; height: 1px;">

#### Exercice 5: la fonction $\rho$

La fonction $\rho$ définie par : $A[x,y,z] \leftarrow A[x,y,(z-\frac{(t+1)(t+2)}{2}) \mod w]$ avec $0\leq t < 24$ et: $\begin{pmatrix} 0 & 1 \\ 2 & 3 \end{pmatrix}^t \begin{pmatrix} 1 \\ 0 \end{pmatrix} = \begin{pmatrix} x \\ y \end{pmatrix} \mod 5 \quad\text{ tel que: } t = -1 \text{ si } x=y=0$

In [8]:
#la fonction Rho
def rho(A, w):
    # Matrice de transformation
    M = np.array([[0, 1], [2, 3]], dtype=int)
    
    # Parcourir chaque (x, y)
    for x in range(5):
        for y in range(5):
            # Cas spécial : x = y = 0
            if x == 0 and y == 0:
                t = -1
            else:
                # Calculer t tel que M^t * [1, 0] = [x, y] mod 5
                C = np.array([x, y], dtype=int)
                V = np.array([1, 0], dtype=int)
                t = 0
                while True:
                    # Si C = V retoune l'ancienne valeur de t
                    if np.array_equal(V % 5, C % 5):
                        break
                    # Si C != V en multiple par M
                    V = M @ V     # @ matrix cross-product multiplication
                    t += 1
                    if t >= 24:  # Limite pour éviter une boucle infinie
                        raise ValueError("t non trouvé pour (x, y) =", (x, y))
            
            # Calculer le décalage z'
            if t == -1:
                z_shift = 0  # Pas de décalage pour (0, 0)
            else:
                z_shift = (t + 1) * (t + 2) // 2
            
            # Appliquer la rotation pour chaque z
            for z in range(w):
                new_z = (z - z_shift) % w
                A[x, y, z] = A[x, y, new_z]
                #print(f"A[{x},{y}, {z}] -> A[{x},{y},{new_z}]")
    
    return A

#B = chBits_to_state(msg_bin,w)
#np.printoptions(threshold=np.inf)
#print(rho(B,w))


<hr style="color: black; height: 1px;">

#### Exercice 6: la fonction $\pi$

Pour $(x,y) \in \mathbb{F}_5 ^2, (x',y')$ est donnée par : $\begin{pmatrix} x' \\ y'\end{pmatrix} = \begin{pmatrix} 0 & 1 \\ 2 & 3 \end{pmatrix}^t \begin{pmatrix} x \\ y\end{pmatrix} \mod 5$ . Comme $A[x', y', z] \leftarrow A[x,y,z]$

In [9]:
def pi(A, w): 
    # Matrice de transformation
    M = np.array([[0, 1], [2, 3]], dtype=int)
   
    # Parcourir chaque (x, y)
    for x in range(5):
        for y in range(5):
            # Calculer (x', y') = M * (x, y) mod 5
            xy = np.array([x, y], dtype=int)
            xy_new = M @ xy % 5
            x_new, y_new = xy_new
            
            # Copier A[x, y, z] vers A_new[x', y', z]
            A[x_new, y_new, :] = A[x, y, :]

            #Pour voir les resultat effacer les commentaires pour 2 lignes 
            #for z in range(w):
                #print(f"A[{x}, {y},{z}] => A_new[{x_new}, {y_new},{z}]")
    return A
    
#B = chBits_to_state(msg_bin,w)
#pi(B,w)

<hr style="color: black; height: 1px;">

#### Exercice 7: la fonction $\chi$

$A[x,y,z] \leftarrow A[x,y,z] \oplus \left(\overline{A}[(x+1)\mod 5,y,z]\wedge A[(x+2)\mod 5,y,z]\right)$. avec $\overline{A}[x,y,z]=(A[x,y,z]+1)= Not(A[x,y,z])$

In [10]:
def chi(A, w):
    for x in range(5):
        for y in range(5):
            for z in range(w):
                #    A[x,y,z] =   A[x,y,z] XOR (A[x+1,y,z] + 1)            & A[x+2,y,z]
                A[x, y, z] = A[x, y, z] ^ ((A[(x + 1) % 5, y, z] ^ 1) & A[(x + 2) % 5, y, z])
    return A

#B = chBits_to_state(msg_bin,w)
#chi(B,w)

<hr style="color: black; height: 1px;">

#### Exercice 8: la foction $\iota$

$A[0, 0, z] = A[0, 0, z] \oplus RC[z]$ .


In [24]:
def iota(A, Nr):
    
    
    # Constantes RC pour les 24 tours
    RC = [0x0000000000000001, 0x0000000000008082, 0x800000000000808A, 0x8000000080008000, 0x000000000000808B, 0x0000000080000001,
          0x8000000080008081, 0x8000000000008009, 0x000000000000008A, 0x0000000000000088, 0x0000000080008009, 0x000000008000000A,
          0x000000008000808B, 0x800000000000008B, 0x8000000000008089, 0x8000000000008003, 0x8000000000008002, 0x8000000000000080,
          0x000000000000800A, 0x800000008000000A, 0x8000000080008081, 0x8000000000008080, 0x0000000080000001, 0x8000000080008008
         ]

    #Si t>=24 erreur
    if Nr < 0 or Nr >= len(RC):
        raise ValueError("L'indice de tour doit etre entre 0 et 23!")
    
    # Récupérer et Appliquer la constante RC à A[0, 0]
    rc = np.uint64(RC[Nr])
    A[0, 0] = np.logical_xor(A[0,0], rc)

    return A

#B = chBits_to_state(msg_bin,w)
#print(iota(B,4))

<hr style="color: black; height: 1px;">

#### Exercice 9: la fonction eponge $f$

In [25]:
# La fonction keccak-f
def keccak_f(state, w):
    for rnd in range(24):
        # Étape 1: Theta
        state = theta(state, w)
        
        # Étape 2: Rho
        state = rho(state, w)
        
        # Étape 3: Pi
        state = pi(state, w)
        
        # Étape 4: Chi
        state = chi(state, w)
        
        # Étape 5: Iota
        state = iota(state, rnd)
    
    return state

#Phase d'absorption du sponge 
def absorb(msg, r, c, w):
    # Initialiser l'état par des zéros
    state = np.zeros((5, 5, w), dtype=np.uint64)
    
    # Diviser le msg en blocs de taille r
    block_size = r
    for i in range(0, len(msg), block_size):
        block = msg[i:i+block_size].ljust(block_size, '0')
        
        # Convertir le bloc en état
        block_state = chBits_to_state(block, w)
        
        # Assurer que block_state est un tableau numpy de type np.uint64
        block_state = np.array(block_state, dtype=np.uint64)
        
        # XOR le bloc avec l'état courant (seulement la partie r)
        for x in range(5):
            for y in range(5):
                for z in range(w):
                    if (x + 5 * y) * w + z < r:
                        state[x, y, z] ^= block_state[x, y, z]
        
        # Appliquer la permutation Keccak-f
        state = keccak_f(state, w)
    
    return state

#Phase d'extraction du sponge 
def squeeze(state, output_length, r, w):

    output = ""
    while len(output) < output_length:
        # Extraire les premiers 'r' bits de l'état
        for x in range(5):
            for y in range(5):
                for z in range(w):
                    if (x + 5 * y) * w + z < r:
                        output += str(state[x, y, z])
        
        # Si nous avons besoin de plus de bits, appliquer Keccak-f et continuer
        if len(output) < output_length:
            state = keccak_f(state, w)
    
    return output[:output_length]

#Fonction sponge pour SHA-3.
def sponge(msg, output_length, r, c, w=64):
    # Convertir le msg en bits
    msg_bits = str_to_bin(msg)
    
    # Appliquer le padding
    padded_msg = second_padd(msg_bits, r)
    
    # Phase d'absorption
    state = absorb(padded_msg, r, c, w)
    
    # Phase d'extraction
    output = squeeze(state, output_length, r, w)
    
    return output

#### Exercice 10: Implementation des fonctions de hachage SHA3-x

In [27]:
#Fonction de hachage SHA3-224
def sha3_224(msg):
    out_len = 224
    r = 1152  # 1600 - 2*224
    c = 448  # 2*224
    return sponge(msg, out_len, r, c)

#Fonction de hachage SHA3-256
def sha3_256(msg):
    out_len = 256
    r = 1088  # 1600 - 2*256
    c = 512  # 2*256
    return sponge(msg, out_len, r, c)

#Fonction de hachage SHA3-384
def sha3_384(msg):
    out_len = 384
    r = 832  # 1600 - 2*384
    c = 768  # 2*384
    return sponge(msg, out_len, r, c)

#Fonction de hachage SHA3-512
def sha3_512(msg):
    out_len = 512
    r = 576  # 1600 - 2*512
    c = 1024  # 2*512
    return sponge(msg, out_len, r, c)


### Test

In [29]:
def test_sha3():
    # Demander à l'utilisateur de saisir un message
    msg = input("Entrez un message à hacher avec SHA3 : ")
    
    # En-tête du tableau des résultats
    print("_"*100)
    print("| Algorithme | Hachage                                                        ")
    print("_"*100)
    
    # Calculer les hachages pour le message de l'utilisateur
    hash224 = hex_digest(sha3_224(msg))
    hash256 = hex_digest(sha3_256(msg))
    hash384 = hex_digest(sha3_384(msg))
    hash512 = hex_digest(sha3_512(msg))
    
    # Afficher les résultats
    print(f"| SHA3-224   | {hash224} |")
    print(f"| SHA3-256   | {hash256} |")
    print(f"| SHA3-384   | {hash384} |")
    print(f"| SHA3-512   | {hash512} |")

# Exécuter les tests
if __name__ == "__main__":
    test_sha3()

Entrez un message à hacher avec SHA3 :  secret


____________________________________________________________________________________________________
| Algorithme | Hachage                                                        
____________________________________________________________________________________________________
| SHA3-224   | 8a145ef5cce7c15cfa17cbffb2399dea114816886c26c064062eed5100 |
| SHA3-256   | ded9d6057478444ca1bff0319c8a26d300804024020120128b6925a4bfa4226f00 |
| SHA3-384   | 6d5e4565f4263852e3141e4f6966fbc8ffffffffffffffffbc0e3ce9e3979293708be20650993f5bafae7ec7777ef7a500 |
| SHA3-512   | c7797979622c50aa883ebeda00f12bd2b17548bbc5affbd4fed8dbdd5cd533a5d59ff2d671c99737169be6e29369b67e46d9146731ecf19657bd44a31549fe5800 |


### Resultats du test

Entrez un message à hacher avec SHA3 :  secret

____________________________________________________________________________________________________
| Algorithme | Hachage                                                        
____________________________________________________________________________________________________
| SHA3-224   | 8a145ef5cce7c15cfa17cbffb2399dea114816886c26c064062eed5100 | 

| SHA3-256   | ded9d6057478444ca1bff0319c8a26d300804024020120128b6925a4bfa4226f00 |

| SHA3-384   | 6d5e4565f4263852e3141e4f6966fbc8ffffffffffffffffbc0e3ce9e3979293708be20650993f5bafae7ec7777ef7a500 |

| SHA3-512   |c7797979622c50aa883ebeda00f12bd2b17548bbc5affbd4fed8dbdd5cd533a5d59ff2d671c99737169be6e29369b67e46d9146731ecf19657bd44a31549fe5800 |
